# Simple SEC Discord Bot

A Discord bot that monitors and alerts you about new SEC filings in real-time. You can configure it to track specific form types (like Forms 3, 4, 5) and companies by their CIK numbers.

> **Note**: On startup, the bot will process that day's backlog of filings, which may take some time.

## Setting up the Discord Bot

1. Go to [Discord Developer Portal](https://discord.com/developers/applications)
2. Click "New Application" and give it a name (e.g., "sec-bot")
3. Navigate to the "Bot" section and create a bot token
4. Copy the token - you'll need to insert it as `TOKEN` in the configuration
5. Under "Privileged Gateway Intents", enable "Message Content Intent"
6. Go to OAuth2 section and configure the following:
   - Select "bot" under Scopes
   - Enable required permissions:
     - Send Messages
     - Read Message History
7. Generate the OAuth2 URL and use it to invite the bot to your server

## Enabling Developer Mode on Discord

To get channel IDs, you need to enable Developer Mode:

1. Open Discord
2. Go to User Settings → App Settings → Advanced
3. Enable Developer Mode
4. Right-click any channel and select "Copy ID" to get the channel ID

## Configuration

### Basic Configuration

Edit the following variables in the code:

```python
# Replace with your bot token
TOKEN = 'YOUR_DISCORD_BOT_TOKEN'

# Configure channels and their filters
CHANNEL_FILTERS = {
    1234567890: {  # Replace with your channel ID
        'forms': None,  # None means all forms
        'ciks': None   # None means all companies
    }
}
```

### Advanced Channel Filtering

You can set up multiple channels with different filtering rules:

```python
CHANNEL_FILTERS = {
    1234567890: {  # Channel for all filings
        'forms': None,
        'ciks': None
    },
    9876543210: {  # Channel for Forms 3,4,5 only
        'forms': ['3','4','5'],
        'ciks': None
    },
    1122334455: {  # Channel for specific company (e.g., Tesla)
        'forms': None,
        'ciks': [320193]
    }
}
```

## Bot Commands

The bot supports the following commands:

- `!add_filter`: Add a filter for the current channel
  ```
  !add_filter "3,4,5" "320193,789019"
  ```
  First argument is form types, second is CIK numbers. Both are optional.

- `!show_filter`: Display current channel's filter settings
  ```
  !show_filter
  ```

- `!clear_filter`: Remove all filters from the current channel
  ```
  !clear_filter
  ```

## Finding Company CIK Numbers

1. Visit [SEC's Company Search](https://www.sec.gov/edgar/search/)
2. Search for the company
3. The CIK number will be displayed in the search results

## Running the Bot

1. Configure your `TOKEN` and `CHANNEL_FILTERS`
2. Run the bot:
```bash
python bot.py
```

## Example Output

![sec bot](static/sec-bot-1.png)

## Troubleshooting

- If the bot doesn't respond, check if the token is correct
- Ensure the bot has proper permissions in the Discord server
- Verify the channel IDs in `CHANNEL_FILTERS` are correct
- Check console output for any error messages


In [ ]:
import discord
from discord.ext import commands, tasks
from datamule import Monitor
import asyncio
import re
import nest_asyncio

# Apply nest_asyncio to make async work in Jupyter
nest_asyncio.apply()


# USER CONFIGURATION BELOW ###############################################

# Bot configuration
TOKEN = 'YOUR_DISCORD_BOT_TOKEN'

# Channel filters
CHANNEL_FILTERS = {
    1318840948305956918: {  # Default channel
        'forms': None,  # None means all forms
        'ciks': None    # None means all companies
    },
    1320808319358734436 : {  # Form 3,4,5 channel
        'forms': ['3','4','5'],
        'ciks': None
    },
    1320808560149528636 : {  # You can also filter by CIK, here is an example for tesla.
        'forms': None,
        'ciks': [320193]
    }
}

# USER CONFIGURATION ABOVE ###############################################

# Bot setup
intents = discord.Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix='!', intents=intents)

class SECMonitor:
    def __init__(self, bot):
        self.bot = bot
        self.monitor = Monitor()
        self.is_running = False

    def process_filing(self, filing):
        """Process a single SEC filing and return formatted data."""
        source = filing['_source']
        cik = int(source['ciks'][0])
        acc_no_match = re.match(r'(\d+)-(\d+)-(\d+):', filing['_id'])
        acc_no_with_dashes = f"{acc_no_match[1]}-{acc_no_match[2]}-{acc_no_match[3]}"
        acc_no_no_dashes = f"{acc_no_match[1]}{acc_no_match[2]}{acc_no_match[3]}"
        
        url = (f"https://www.sec.gov/Archives/edgar/data/{cik}/"
               f"{acc_no_no_dashes.zfill(18)}/"
               f"{acc_no_with_dashes}-index.html")
        
        return {
            'url': url,
            'display_name': source['display_names'][0],
            'filing_date': source['file_date'],
            'submission_type': source['file_type'],
            'cik': cik
        }

    def should_send_to_channel(self, channel_id, filing):
        """Check if filing should be sent to this channel based on filters."""
        if channel_id not in CHANNEL_FILTERS:
            return False
            
        filters = CHANNEL_FILTERS[channel_id]
        
        # Check form type filter
        if filters['forms'] is not None:
            if filing['submission_type'] not in filters['forms']:
                return False
                
        # Check CIK filter
        if filters['ciks'] is not None:
            if filing['cik'] not in filters['ciks']:
                return False
                
        return True

    async def send_to_discord(self, filings):
        """Handle new filings from monitor."""
        try:
            for filing in filings:
                processed = self.process_filing(filing)
                
                # Send to each channel based on filters
                for channel_id in CHANNEL_FILTERS.keys():
                    if self.should_send_to_channel(channel_id, processed):
                        channel = self.bot.get_channel(channel_id)
                        if channel:
                            embed = discord.Embed(title=f"New: {processed['submission_type']}")
                            embed.add_field(name="Company", value=processed['display_name'])
                            embed.add_field(name="Filing Date", value=processed['filing_date'])
                            embed.add_field(name="URL", value=processed['url'])
                            await channel.send(embed=embed)
                            
        except Exception as e:
            print(f"Error processing submissions: {str(e)}")

    async def start_monitoring(self):
        """Start the SEC filing monitor."""
        if not self.is_running:
            self.is_running = True
            try:
                await self.monitor._monitor(
                    callback=self.send_to_discord,
                    poll_interval=1000,
                    quiet=True
                )
            except Exception as e:
                print(f"Monitor error: {str(e)}")
                self.is_running = False

# Bot commands
@bot.command(name='add_filter')
async def add_filter(ctx, forms: str = None, ciks: str = None):
    """Add a filter for the current channel. Example: !add_filter "3,4,5" "320193,789019" """
    channel_id = ctx.channel.id
    
    # Parse forms
    form_list = None
    if forms:
        form_list = [f.strip() for f in forms.split(',')]
        
    # Parse CIKs
    cik_list = None
    if ciks:
        cik_list = [int(cik.strip()) for cik in ciks.split(',')]
    
    CHANNEL_FILTERS[channel_id] = {
        'forms': form_list,
        'ciks': cik_list
    }
    
    await ctx.send(f"Filter set for channel {channel_id}:\nForms: {form_list}\nCIKs: {cik_list}")

@bot.command(name='show_filter')
async def show_filter(ctx):
    """Show the current channel's filter settings."""
    channel_id = ctx.channel.id
    if channel_id in CHANNEL_FILTERS:
        filters = CHANNEL_FILTERS[channel_id]
        await ctx.send(f"Current filters:\nForms: {filters['forms']}\nCIKs: {filters['ciks']}")
    else:
        await ctx.send("No filters set for this channel.")

@bot.command(name='clear_filter')
async def clear_filter(ctx):
    """Clear all filters for the current channel."""
    channel_id = ctx.channel.id
    if channel_id in CHANNEL_FILTERS:
        del CHANNEL_FILTERS[channel_id]
        await ctx.send("Filters cleared for this channel.")
    else:
        await ctx.send("No filters were set for this channel.")

# Bot events
@bot.event
async def on_ready():
    print(f'Bot connected as {bot.user}')
    monitor = SECMonitor(bot)
    # Start monitoring in the background
    bot.loop.create_task(monitor.start_monitoring())

# Start the bot
def run_bot():
    """Run the Discord bot."""
    try:
        bot.run(TOKEN)
    except KeyboardInterrupt:
        print("Bot stopped by user")
    except Exception as e:
        print(f"Bot error: {str(e)}")

if __name__ == "__main__":
    run_bot() 